In [1]:
%autosave 0

Autosave disabled


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install ortools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 81.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 34.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3


In [2]:
import os
import pandas as pd
import numpy as np
import re
import collections
import pickle
from ast import literal_eval
import random
import scipy
import math
import datetime
import subprocess

from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

#Path and Files

In [3]:
# path for the 'Data' folder provided by DP
# global vars

dir_loc = '/content/drive/Shareddrives/Private Unlimited Drive #1/DDS/Analytics Project/AOP_DP_Analytics/Data'

path = '/content/drive/Shareddrives/Private Unlimited Drive #1/DDS/Analytics Project/AOP_DP_Analytics/'

In [4]:
# upload files
all_original = pickle.load(open("/content/drive/Shareddrives/Private Unlimited Drive #1/DDS/Analytics Project/Coding/Results/all_original_complete.p", "rb"))
all_default_tours = pickle.load(open("/content/drive/Shareddrives/Private Unlimited Drive #1/DDS/Analytics Project/Coding/Results/all_default_tours.p", "rb"))

#Functions

In [5]:
# create dataframes for a region everyday

def generate_region_volume(region_name):
  # path of folder
  dir_region_instances = dir_loc + '/Instances/' + region_name
  dir_region_volume = dir_loc + '/Volumes/'
  # dir_region_districts = dir_region_instances + '/Districts'

  # load post_object-to-route_pos_id and post_point-to-post_object files 
  po_file = pd.read_csv(dir_region_instances + '/post_order_id_mapping.dat', sep='\t', names=('PostObjectId', 'RoutePosID'))
  pp_file = pd.read_csv(dir_region_instances + '/post_point_information.dat', sep='\t', names=('PostPointId', 'PostObjectId'))

  # pp_file adjustment by splitting list of post object ids
  pp_file['PostObjectId'] = pp_file['PostObjectId'].apply(literal_eval)
  pp_file = pp_file.explode('PostObjectId', ignore_index=True)

  # complete  post object list-district mapping
  region_district_df = generate_region_district(region_name = region_name)
  region_district_df.rename(columns = {'PostObjectID' : 'PostObjectId'}, inplace = True)

  # list of volume file paths
  day_names = []
  vol_path_list = []
  vol_day_map = {}
  day_map = {'mo' : 'Monday',
             'di' : 'Tuesday',
             'mi' : 'Wednesday',
             'do' : 'Thursday',
             'fr' : 'Friday',
             'sa' : 'Saturday'}

  for filename in os.listdir(dir_region_volume):
    vol_path_list.append(dir_region_volume + filename)
    day = filename[-6:-4]
    vol_day_map[filename] = day_map[day]

  # store dataframes of a region, map with post object id
  region_vol_day_dict = {}
  for vol_path in vol_path_list:
    vol_df = pd.read_csv(vol_path, sep = ';')
    vol_df.rename(columns = {'BRIEFE' : 'LETTERS',
                             'PAKETE' : 'PACKAGES',
                             'SONSTIGE' : 'OTHERS',
                             'ROUTEPOS_ID' : 'RoutePosID'},
                  inplace = True)
    
    # combining files to a complete table for a region
    vol_po_df = pd.merge(po_file, vol_df, on = 'RoutePosID', how = 'left')
    vol_po_df = pd.merge(pp_file, vol_po_df, on='PostObjectId', how='right')
    vol_po_df = pd.merge(region_district_df, vol_po_df, on='PostObjectId', how='right')

    # store dataframes in dict
    day_key = vol_day_map[vol_path[-18:]]
    region_vol_day_dict[day_key] = vol_po_df
    
  return region_vol_day_dict

In [6]:
def generate_region_district(region_name):
  district_path = dir_region_instances = dir_loc + '/Instances/' + region_name + '/Districts'
  region_districts_list = []

  for filename in os.listdir(district_path):
    file_var = re.sub('.dat', '', filename)
    file_district = pd.read_csv(district_path + '/' + filename, sep='\t', skiprows = [0,1],
                                names=('PostObjectID', 'dum_1', 'dum_2', 'dum_3', 'dum_4', 'dum_5')
                                # usecols = [0]
                                )
    file_district['district'] = file_var
    region_districts_list.append(file_district)

  region_districts_df = pd.concat(region_districts_list, ignore_index = True)
  region_districts_df.drop(['dum_1', 'dum_2', 'dum_3', 'dum_4', 'dum_5'], axis = 1, inplace = True)
  
  return region_districts_df

In [7]:
# generate instances
# for equal shipment scenario


def generate_instances_equal_shipment(region_vol_day, scenario_type, scenario_method, growth_factor):
  df = region_vol_day.copy() #.copy() used to avoid recopying on the original dataframe
  # df = region_vol_day

  df['scenario_letters'] = df['LETTERS'].apply(lambda x :scenario_type(pos_delivery = x, method = scenario_method, rate_pct = growth_factor ))
  df['scenario_packages'] = df['PACKAGES'].apply(lambda x :scenario_type(pos_delivery = x, method = scenario_method, rate_pct = growth_factor))
  df['scenario_others'] = df['OTHERS'].apply(lambda x :scenario_type(pos_delivery = x, method = scenario_method, rate_pct = growth_factor))

  df['scenario_all'] = df['scenario_letters'] + df['scenario_packages'] + df['scenario_others']

  return df

In [8]:
# generate instances
# for unequal shipment scenario


# def generate_instances_unequal_shipment(region_vol_day, scenario_type, scenario_method, scenario_number, growth_factor):
#   # df = region_vol_day.copy() #.copy() used to avoid recopying on the original dataframe
#   df = region_vol_day

#   sce_letters = 'scenario_' + str(scenario_number) + '_letter'
#   sce_packages = 'scenario_' + str(scenario_number) + '_package'
#   sce_others = 'scenario_' + str(scenario_number) + '_others'

#   df[sce_letters] = df['LETTERS'].apply(lambda x :scenario_type(pos_delivery = x, method = scenario_method, rate_pct = growth_factor ))
#   df[sce_packages] = df['PACKAGES'].apply(lambda x :scenario_type(pos_delivery = x, method = scenario_method, rate_pct = growth_factor))
#   df[sce_others] = df['OTHERS'].apply(lambda x :scenario_type(pos_delivery = x, method = scenario_method, rate_pct = growth_factor))

#   sce_all = 'scenario_' + str(scenario_number) + '_all'
#   df[sce_all] = df[sce_letters] + df[sce_packages] + df[sce_others]

#   return df

In [9]:
# generate real instances based on poisson

def random_poisson_instances(pos_delivery, method, rate_pct):
  rate = rate_pct/100
  rng = np.random.default_rng()
  poisson_dist = rng.poisson(lam = pos_delivery * (1 + rate), size = 52)

  if method == 'random':
    return random.choice(poisson_dist)
  if method == 'mode':
    return scipy.stats.mode(poisson_dist, keepdims = True)[0][0]
  else:
    return 'only options : [random, mode]'

In [10]:
# get start and end node for a district

def start_end_points(region_name, district):
    # get start and end node
    district_path = dir_loc + '/Instances/' + region_name + '/Districts/' + district + '.dat'
    rows_needed = [1]
    file_district = pd.read_csv(district_path, sep='\t', skiprows = lambda x : x not in rows_needed,
                                names=('dum_0', 'start_point', 'end_point', 'dum_1', 'dum_2', 'dum_3', 'dum_4', 'dum_5', 'dum_6')
                                )
    start_point = file_district['start_point'][0]
    end_point = file_district['end_point'][0]

    points = [start_point, end_point]

    return points

In [11]:
# generate distance matrix and mapping for a scenario

def generate_distance_matrix_map(region_name, df_day_district, points, sce_col, district):
      scenario = sce_col

      col_use = ['PostPointId']
      col_use.append(scenario)
      df_day_district_scenario = df_day_district[col_use]
      
      # removing nodes with zero demand
      df_day_district_scenario_filtered = df_day_district_scenario[df_day_district_scenario[scenario] != 0]
      pp_id_day_district_scenario = df_day_district_scenario_filtered['PostPointId'].unique().tolist()
      
      # add start and end node if not in node list yet
      for point in points:
        if point not in pp_id_day_district_scenario:
          pp_id_day_district_scenario.append(point)

      # get distance file and dataframes
      distance_path = dir_loc + '/Instances/' + region_name + '/distances'
      district_distance = pd.read_csv(distance_path + '/distances_' + district + '.dat',
                                  names=['pp_1', 'pp_2', 'dist']
                                  )
      # remove unused postpoints
      district_distance_filtered = district_distance[(district_distance['pp_1'].isin(pp_id_day_district_scenario)) & (district_distance['pp_2'].isin(pp_id_day_district_scenario))]
      distance_matrix_df = district_distance_filtered.pivot(index = 'pp_1', columns = 'pp_2', values = 'dist')

      # generate and revise start and end nodes
      points = start_end_points(region_name = region_name, district = district)
      distance_matrix_df[points[0]][points[1]] = 0
      distance_matrix_df[points[1]][points[0]] = 0

      # distance matrix
      distance_matrix_array = distance_matrix_df.to_numpy()
      distance_matrix_array = distance_matrix_array.tolist()

      # node mapping
      map_val = list(range(0, len(distance_matrix_df)))
      nodes = distance_matrix_df.index.values.tolist()
      mapping = dict(zip(map_val, nodes))

      return distance_matrix_array, mapping

In [12]:
def create_dm_map_equal_shipment(region, data, save_loc, rate_list, instance_variations):
  i = 1
  # print(region)
  dir_region = save_loc + '/' + region
  if not os.path.exists(dir_region):
    os.mkdir(dir_region)


  # filter by day
  result_day = {}
  for day in data.keys():
    # print(day)
    df_day = data[day]

    # filter by rate
    result_rate = {}
    for rate in rate_list:
      # print(rate)
      df_day_rate = df_day[rate]

      # filter by instance number
      result_instance_number = {}
      for instance_counter in range(1, instance_variations + 1):
        df_day_rate_instance = df_day_rate[instance_counter]

        # filter by district
        result_district = {}
        for district in df_day_rate_instance['district'].value_counts().index.tolist():
          df_day_district = df_day_rate_instance[df_day_rate_instance['district'] == district]

          scenario_list = df_day_district.columns.tolist()[7:]

          # get start and end node
          points = start_end_points(region_name = region, district = district)
          
          # filter by scenario
          result_mail_demand = {}
          volume = {}
          for scenario in scenario_list:
            mail_type = scenario.split('_')[1]
            # get volume
            volume[mail_type] = df_day_district[scenario].sum()

            # dir of mail
            # mail_names = ['letter', 'package', 'others', 'all']
            
            # generate mapping and distance matrix only for 'all'
            if 'all' in scenario:
              distance_matrix, mapping = generate_distance_matrix_map(region_name = region, df_day_district = df_day_district,
                                                                      points = points, sce_col = scenario, district = district)
              distance_matrix_name = 'dm_' + region + '_' + day + '_' + district + '_' + str(rate) + '_' + str(instance_counter)
              mapping_name = 'map_' + region + '_' + day + '_' + district + '_' + str(rate) + '_' + str(instance_counter)
              pickle.dump(distance_matrix, open(dir_region + '/' + "%s.p"%distance_matrix_name, "wb"))
              pickle.dump(mapping, open(dir_region + '/' + "%s.p"%mapping_name, "wb"))

          # log update
            # print(i, ':', distance_matrix_name, ':', distance_matrix.shape)
          print(i, ':', distance_matrix_name, ':', len(distance_matrix))
          i = i+1

          result_end = {}
          result_end['dm'] = distance_matrix
          result_end['map'] = mapping
          result_end['start_end_points'] = points
          result_end['volume'] = volume
        
          result_mail_demand['all'] = result_end
          result_district[district] = result_mail_demand
        result_instance_number[instance_counter] = result_district
      result_rate[rate] = result_instance_number
    result_day[day] = result_rate

  return result_day

In [13]:
# arrange sequence of tour

def arrange_tour(solver_tour, start_end_points):

    #Get the start and end point of the specific district of the instance
    start_PP, end_PP = start_end_points 
    arranged_tour = 0

    #FUNCTION TO REARRANGE THE LIST WITH CASE OF SAME START AND END POSTPOINT
    def rearrange_list(numbers, specific_value):
        # Find the index of the specific value in the list
        index = numbers.index(specific_value)
        
        # Check if the chosen element is repeated twice consecutively
        if numbers[index] == numbers[index + 1]:
            # Split the list into two parts based on the index
            part1 = numbers[:index]
            part2 = numbers[index + 1:]
            
            # Rearrange the list by concatenating the parts and adding the specific value at the beginning and end
            rearranged_list = part2 + part1 + [specific_value]
        else:
            # Split the list into two parts based on the index
            part1 = numbers[:index]
            part2 = numbers[index:]
            
            # Rearrange the list by concatenating the parts in reverse order
            rearranged_list = part2 + part1
        
        return rearranged_list
    
    
    #FUNCTION TO REARRANGE THE LIST WITH CASE OF DIFFERENT START AND END POSTPOINT
    def rearrange_circular_tour(tour, start_point, end_point):
    # Find the index of the start point in the tour list
        start_index = tour.index(start_point)
            
        # Find the index of the end point in the tour list
        end_index = tour.index(end_point)
                
        # Check if the start and end points are already at the first and last positions
        if start_index == 0 and end_index == len(tour) - 1:
            return tour
                
        # Split the tour list into two parts based on the start and end indices
        if start_index < end_index:
            part1 = tour[0:start_index + 1]
            part1.reverse()

            part2 = tour[start_index + 1:]
            part2.reverse()
                
        else:
            part1 = tour[start_index:] + tour[:end_index+1]
            part2 = tour[end_index+1:start_index]
                
        # Rearrange the tour list by concatenating the parts
        rearranged_tour = part1 + part2
        
        return rearranged_tour
    


    #Case of start and end point equal
    if start_PP == end_PP:
        #If the start and end point are already in correct positions return list as it is                                                                              
        if start_PP == solver_tour[0] and end_PP == solver_tour[len(solver_tour)-1]:                    
            arranged_tour = solver_tour
        else: 
            #Check if start or end point (doesn't matter they have the same value) appears less than two times on the list
            if list.count(solver_tour, start_PP) < 2:
                #If if is not repeated. Check if at least start_point is in first_position.                                                   
                if start_PP == solver_tour[0]: 
                    #If start_point is in first position just append end point at the end of list.                                                         
                    arranged_tour = solver_tour + [end_PP]
                else: 
                    #Rearrange the tour first with start_PP at the beginning of the list before appending end_PP
                    arranged_tour = rearrange_list(solver_tour, start_PP)
                    arranged_tour = arranged_tour + [end_PP]               
            else: 
                #Rearrange the tour in the correct order
                arranged_tour = rearrange_list(solver_tour, start_PP)
    #Case of start and end point different
    else:
       arranged_tour = rearrange_circular_tour(tour = solver_tour, start_point = start_PP, end_point = end_PP)
    
    return arranged_tour

In [14]:
#FUNCTION TO GET SUBSET OF A COMPLETE TOUR AND ITS LENGTH KEEPING SEQUENCE BASED ON INSTANCES VOLUMES

#INPUTS:    Complete Tour (DP or Concord), Complete Tour info (DP or Concord), 
#           Distance Matrix for the instance (generated from instance generation function), Mapping of PostPoints Needed (Dictionary obtained from instance generation function)
#OUTPUTS:   Tour Sequence Fixed, Tour Length, Difference between New Tour and Complete Tour



def new_tour_sequence_fixed(complete_tour, dm_instance, mapping_pp_needed):
    #get the list of only the Postpoint needed for the specific instance
    list_pp_needed = list(mapping_pp_needed.values())

    #Keep from complete_tour only the needed PostPoints according to the list of PostPoints needed mantaining sequence
    tour_seq_fixed = [x for x in complete_tour if x in list_pp_needed]

    #Generate a dataframe from the distance matrix with the correct row-column combination based on PostPoints required
    df_dm_instance = pd.DataFrame(dm_instance)
    df_dm_instance = (df_dm_instance.rename(columns= mapping_pp_needed)).rename(index = mapping_pp_needed)

    #FUNCTION to compute length of new tour (Adapting using logic of function already created for default_route calculation: get_tour_length())
    
    def get_seq_tour_length(sequence_list, distance_df):
        tour_length = 0 
        for i in range(len(sequence_list)-1):
            # Specify the specific values for row and column in the dataframe
            PostPoint_predecesor = sequence_list[i]
            PostPoint_succesor = sequence_list[i+1]

            # Get the distance value needed based on the Post Point combination or raise error
            try:
                distance_value = distance_df[PostPoint_predecesor][PostPoint_succesor]
            except: 
                print('ERROR: No distance for PostPoint combination')

            # Aggreate sum value until end of loop to compute total tour length
            tour_length += distance_value
            
        return round(tour_length,2)
    
    #Calculate tour length of new tour which skips PostPoints
    tour_seq_fixed_length = get_seq_tour_length(tour_seq_fixed, df_dm_instance)
    

    return tour_seq_fixed, tour_seq_fixed_length

In [15]:
# route similarity

def find_similarity_route(Tour1, Tour2):
    predecessors1 = {Tour1[i]: Tour1[i-1] for i in range(1, len(Tour1))}
    predecessors2 = {Tour2[i]: Tour2[i-1] for i in range(1, len(Tour2))}
    num_changed_predecessors = 0

    for node, pred2 in predecessors2.items():
        pred1 = predecessors1[node]
        if pred1 != pred2:
            num_changed_predecessors += 1
    percent_change = num_changed_predecessors / (len(Tour1)-1) * 100
    sim_pct = 100 - percent_change

    return num_changed_predecessors, sim_pct

In [16]:
# heuristic google OR algo

class TSP:
    def __init__(self, distance_matrix):
        #read distance matrix --> list
        self.dist_mat = distance_matrix
        self.dist_mat = self.round_up(self.dist_mat)
        # Create data model
        self.data = self.create_data_model()
        # Create routing index manager
        self.manager = pywrapcp.RoutingIndexManager(len(self.data['distance_matrix']),
                                            self.data['num_vehicles'], self.data['depot'])
        # Create Routing Model
        self.routing = pywrapcp.RoutingModel(self.manager)
        # Define cost of each arc
        self.transit_callback_index = self.routing.RegisterTransitCallback(self.distance_callback)
        self.routing.SetArcCostEvaluatorOfAllVehicles(self.transit_callback_index)
    
    def round_up(self , lst):
        '''
        input : list of lists having non integral values
        output : list of lists having integral values.
        Function to round up numbers and return as integers
        '''
        rounded_lst = []
        for inner_lst in lst:
            rounded_inner_lst = []
            for num in inner_lst:
                rounded_num = int(round(num))
                rounded_inner_lst.append(rounded_num)
            rounded_lst.append(rounded_inner_lst)
        return rounded_lst        


    def create_data_model(self):
        # Stores the data for the problem
        data = {}
        data['distance_matrix'] = self.dist_mat 
        data['num_vehicles'] = 1
        data['depot'] = 0
        return data

    def distance_callback(self, from_index, to_index):
        # Returns the distance between the two nodes
        from_node = self.manager.IndexToNode(from_index)
        to_node = self.manager.IndexToNode(to_index)
        return self.data['distance_matrix'][from_node][to_node]

    def solve(self):
        # Setting first solution heuristic
        search_parameters = pywrapcp.DefaultRoutingSearchParameters()
        search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
        # Solve the problem
        solution = self.routing.SolveWithParameters(search_parameters)
        if solution:
            # Get the route as an array
            route = []
            index = self.routing.Start(0)
            while not self.routing.IsEnd(index):
                node = self.manager.IndexToNode(index)
                route.append(node)
                index = solution.Value(self.routing.NextVar(index))
            # Get the objective value
            obj_value = solution.ObjectiveValue()
            # route = self.map_nodes(route)
            return route, obj_value

In [17]:
def map_actual_route(route, mapping):
  tour_mapped = [mapping[a] for a in route]
  return tour_mapped

# Solver for Complete Tours

In [18]:
all_default_tours.keys()

dict_keys(['Warmsen', 'Uerze', 'Hannover 92'])

In [19]:
all_default_tours['Uerze'].keys()

dict_keys(['31311-22', '31311-42', '31311-62', '31311-02', '31311-81', '31311-05', '31311-51', '31311-25', '31311-61', '31311-21', '31311-24', '31311-03', '31311-01', '31311-08', '31311-04', '31311-31', '31311-07'])

In [20]:
all_default_tours['Uerze']['31311-08'].keys()

dict_keys(['default_route', 'mapping', 'route_cost'])

In [21]:
all_default_tours['Uerze']['31311-08']['route_cost']

22387.52

In [22]:
%%time

all_complete_solver = {}

# iterate through all regions
for region in all_original.keys():
  data_region = all_original[region]
  region_complete_solver = {}
  # iterate through all districts
  for district in data_region.keys():
    start = datetime.datetime.now()
    data_district = data_region[district]
    # solve TSP
    tsp = TSP(data_district['dm'])
    # store and rearrange data
    complete_tour_solver = {}
    complete_tour_solver['tour'], complete_tour_solver['cost'] = tsp.solve()
    complete_tour_solver['tour'] = map_actual_route(complete_tour_solver['tour'], data_district['map'])
    complete_tour_solver['tour'] = arrange_tour(solver_tour = complete_tour_solver['tour'],
                                                start_end_points = data_district['start_end_points'])
    print(region, district, 'solved.', 'solver cost', complete_tour_solver['cost'] ,
          '. default DP cost', all_default_tours[region][district]['route_cost'], '. time needed -->', datetime.datetime.now() - start)
    region_complete_solver[district] = complete_tour_solver
  all_complete_solver[region] = region_complete_solver

Warmsen 31606-13 solved. solver cost 23586 . default DP cost 24318.73 . time needed --> 0:00:06.988763
Warmsen 31606-11 solved. solver cost 31501 . default DP cost 33712.81 . time needed --> 0:00:08.676446
Warmsen 31600-04 solved. solver cost 18515 . default DP cost 19125.47 . time needed --> 0:00:05.138086
Warmsen 31600-03 solved. solver cost 29907 . default DP cost 31612.66 . time needed --> 0:00:10.935547
Warmsen 31600-02 solved. solver cost 22537 . default DP cost 23001.26 . time needed --> 0:00:14.176260
Warmsen 31603-07 solved. solver cost 28848 . default DP cost 30426.08 . time needed --> 0:00:10.315263
Warmsen 31603-08 solved. solver cost 24423 . default DP cost 25470.13 . time needed --> 0:00:09.967033
Warmsen 31604-10 solved. solver cost 29642 . default DP cost 30293.02 . time needed --> 0:00:12.937700
Warmsen 31603-05 solved. solver cost 23047 . default DP cost 24083.81 . time needed --> 0:00:18.162467
Warmsen 31603-06 solved. solver cost 25625 . default DP cost 27047.85 . t

In [23]:
all_complete_solver.keys()

dict_keys(['Warmsen', 'Uerze', 'Hannover 92'])

In [24]:
all_complete_solver['Uerze'].keys()

dict_keys(['31311-31', '31311-07', '31311-04', '31311-01', '31311-25', '31311-22', '31311-03', '31311-02', '31311-24', '31311-42', '31311-61', '31311-51', '31311-21', '31311-05', '31311-62', '31311-08', '31311-81'])

In [25]:
all_complete_solver['Uerze']['31311-05'].keys()

dict_keys(['tour', 'cost'])

In [26]:
all_complete_solver['Uerze']['31311-05']['cost']

21084

In [27]:
pickle.dump(all_complete_solver, open("/content/drive/Shareddrives/Private Unlimited Drive #1/DDS/Analytics Project/Coding/Results/all_complete_solver.p", "wb"))

# Base Data For All Regions

In [64]:
%%time

region_list = ['Warmsen', 'Uerze', 'Hannover 92']

all_regions = {}

for region in region_list:
  region_vol_day_dict = generate_region_volume(region_name = region)
  all_regions[region] = region_vol_day_dict

CPU times: user 26.4 s, sys: 1.29 s, total: 27.7 s
Wall time: 29.8 s


In [65]:
all_regions.keys()

dict_keys(['Warmsen', 'Uerze', 'Hannover 92'])

In [66]:
all_regions['Warmsen'].keys()

dict_keys(['Tuesday', 'Thursday', 'Friday', 'Wednesday', 'Monday', 'Saturday'])

In [77]:
all_regions['Warmsen']['Monday'].head()

,PostObjectId,district,PostPointId,RoutePosID,LETTERS,PACKAGES,OTHERS
0,16,31606-14,16,AD14E08623FF9F5AD2293E7DEDB4F4B1,0.45,0.293333,0.0
1,17,31606-14,17,F48CB0CAB0BC117AE030007F0100574C,0.45,0.293333,0.0
2,18,31606-14,16,F48CB0CAB0C7117AE030007F0100574C,0.45,0.293333,0.0
3,19,31606-14,18,F48CB0CAB0C8117AE030007F0100574C,0.00,0.000000,0.0
4,20,31606-14,19,069AE286923C1800E040400A09131416,0.00,0.000000,0.0


#Equal Shipment Development

In [68]:
# duplicate data
# ensure only copy will be updated

all_regions_equal = all_regions.copy()

In [18]:
# parameter
# how many instances for one specific details of region, district, day

instance_variation_equal = 2

In [19]:
#INPUT Rate of mail change 
# rate in percentage
rate_list = [5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]

##Generate Region-Day Instances

In [97]:
%%time
#Creating the complete dictionary of volumes for all region
all_equal_instances = {}

# iterate through all regions
# for region in ['Uerze']:
for region in all_regions_equal.keys():
  data_region = all_regions_equal[region]
  region_dict = {}
  # iterate through all days
  for day in all_regions_equal[region].keys():
    data_region_day = data_region[day]
    day_dict = {}
    # set instance number
    counter = 1
    # iterate through all rates of increasing demand
    for rate in rate_list:
      # print(rate)
      # print('a')
      rate_dict = {}
      # for multiple instances
      # while counter <= instance_variation_equal:
      for counter in range(1, instance_variation_equal + 1):
        # generate instance
        instance = generate_instances_equal_shipment(region_vol_day = data_region_day,
                                                     scenario_type = random_poisson_instances,
                                                     scenario_method = 'random',
                                                     growth_factor = rate
                                                     )
        # print(instance.head(1))
        print('generated :', region, day, ': demand increase rate', rate, '% :', 'instance number', counter)
        # store data
        rate_dict[counter] = instance
        # print('c', counter)
        # counter = counter + 1
      day_dict[rate] = rate_dict
      # print('r', rate)
    region_dict[day] = day_dict
  all_equal_instances[region] = region_dict

generated : Warmsen Tuesday : demand increase rate 5 % : instance number 1
generated : Warmsen Tuesday : demand increase rate 5 % : instance number 2
generated : Warmsen Tuesday : demand increase rate 10 % : instance number 1
generated : Warmsen Tuesday : demand increase rate 10 % : instance number 2
generated : Warmsen Tuesday : demand increase rate 20 % : instance number 1
generated : Warmsen Tuesday : demand increase rate 20 % : instance number 2
generated : Warmsen Tuesday : demand increase rate 30 % : instance number 1
generated : Warmsen Tuesday : demand increase rate 30 % : instance number 2
generated : Warmsen Tuesday : demand increase rate 40 % : instance number 1
generated : Warmsen Tuesday : demand increase rate 40 % : instance number 2
generated : Warmsen Tuesday : demand increase rate 50 % : instance number 1
generated : Warmsen Tuesday : demand increase rate 50 % : instance number 2
generated : Warmsen Tuesday : demand increase rate 60 % : instance number 1
generated : Wa

In [98]:
all_equal_instances.keys()

dict_keys(['Warmsen', 'Uerze', 'Hannover 92'])

In [99]:
all_equal_instances['Uerze'].keys()

dict_keys(['Tuesday', 'Thursday', 'Friday', 'Wednesday', 'Monday', 'Saturday'])

In [100]:
all_equal_instances['Uerze']['Friday'].keys()

dict_keys([5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100])

In [101]:
all_equal_instances['Uerze']['Friday'][30].keys()

dict_keys([1, 2])

In [106]:
all_equal_instances['Uerze']['Friday'][80][2].head()

,PostObjectId,district,PostPointId,RoutePosID,LETTERS,PACKAGES,OTHERS,scenario_letters,scenario_packages,scenario_others,scenario_all
0,11,31311-07,11,F48CB0C2FDFB117AE030007F0100574C,0.818333,0.105,0.3,0,0,1,1
1,12,31311-07,12,F48CB0C2FDFC117AE030007F0100574C,0.000000,0.000,0.0,0,0,0,0
2,13,31311-07,13,F48CB0C2FDFD117AE030007F0100574C,0.818333,0.105,0.3,1,0,1,2
3,14,31311-07,14,F48CB0C2FDFE117AE030007F0100574C,0.818333,0.105,0.3,3,0,2,5
4,15,31311-07,15,F48CB0C2FDFF117AE030007F0100574C,0.818333,0.105,0.3,2,0,0,2


In [112]:
pickle.dump(all_equal_instances, open("/content/drive/Shareddrives/Private Unlimited Drive #1/DDS/Analytics Project/Coding/Results/all_equal_instances_data.p", "wb"))

## Generate Region-Day-District Instances : Distance Matrix

In [4]:
all_equal_instances = pickle.load(open("/content/drive/Shareddrives/Private Unlimited Drive #1/DDS/Analytics Project/Coding/Results/all_equal_instances_data.p", "rb"))

In [24]:
%%time

all_equal_instances_objects = {}

## iterate through all regions : ##
for region in all_equal_instances.keys():
# for region in ['Warmsen']:
  result_region = create_dm_map_equal_shipment(region = region, data = all_equal_instances[region],
                                               save_loc = '/content/drive/Shareddrives/Private Unlimited Drive #1/DDS/Analytics Project/Coding/',
                                               rate_list = rate_list,
                                               instance_variations = instance_variation_equal)
  all_equal_instances_objects[region] = result_region

1 : dm_Warmsen_Tuesday_31603-06_5_1 : 341
2 : dm_Warmsen_Tuesday_31600-01_5_1 : 294
3 : dm_Warmsen_Tuesday_31606-12_5_1 : 313
4 : dm_Warmsen_Tuesday_31600-02_5_1 : 298
5 : dm_Warmsen_Tuesday_31604-09_5_1 : 300
6 : dm_Warmsen_Tuesday_31603-07_5_1 : 322
7 : dm_Warmsen_Tuesday_31604-10_5_1 : 297
8 : dm_Warmsen_Tuesday_31600-03_5_1 : 267
9 : dm_Warmsen_Tuesday_31603-05_5_1 : 311
10 : dm_Warmsen_Tuesday_31603-08_5_1 : 282
11 : dm_Warmsen_Tuesday_31606-11_5_1 : 264
12 : dm_Warmsen_Tuesday_31606-14_5_1 : 247
13 : dm_Warmsen_Tuesday_31606-13_5_1 : 230
14 : dm_Warmsen_Tuesday_31600-04_5_1 : 222
15 : dm_Warmsen_Tuesday_31603-06_5_2 : 332
16 : dm_Warmsen_Tuesday_31600-01_5_2 : 287
17 : dm_Warmsen_Tuesday_31606-12_5_2 : 309
18 : dm_Warmsen_Tuesday_31600-02_5_2 : 288
19 : dm_Warmsen_Tuesday_31604-09_5_2 : 299
20 : dm_Warmsen_Tuesday_31603-07_5_2 : 323
21 : dm_Warmsen_Tuesday_31604-10_5_2 : 279
22 : dm_Warmsen_Tuesday_31600-03_5_2 : 293
23 : dm_Warmsen_Tuesday_31603-05_5_2 : 311
24 : dm_Warmsen_Tues

In [26]:
all_equal_instances_objects.keys()

dict_keys(['Warmsen'])

In [29]:
all_equal_instances_objects['Warmsen'].keys()

dict_keys(['Tuesday', 'Thursday', 'Friday', 'Wednesday', 'Monday', 'Saturday'])

In [31]:
all_equal_instances_objects['Warmsen']['Monday'].keys()

dict_keys([5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100])

In [32]:
all_equal_instances_objects['Warmsen']['Monday'][80].keys()

dict_keys([1, 2])

In [33]:
all_equal_instances_objects['Warmsen']['Monday'][80][2].keys()

dict_keys(['31603-06', '31600-01', '31606-12', '31600-02', '31604-09', '31603-07', '31604-10', '31600-03', '31603-05', '31603-08', '31606-11', '31606-14', '31606-13', '31600-04'])

In [34]:
all_equal_instances_objects['Warmsen']['Monday'][80][2]['31604-09'].keys()

dict_keys(['all'])

In [35]:
all_equal_instances_objects['Warmsen']['Monday'][80][2]['31604-09']['all'].keys()

dict_keys(['dm', 'map', 'start_end_points', 'volume'])

In [43]:
all_equal_instances_objects['Warmsen']['Tuesday'][80][2]['31604-09']['all']['volume']

{'letters': 705, 'packages': 171, 'others': 307, 'all': 1183}

In [40]:
all_equal_instances_objects['Warmsen']['Friday'][5][1]['31600-04']['all']['volume']

{'letters': 317, 'packages': 58, 'others': 162, 'all': 537}

In [44]:
len(all_equal_instances_objects['Warmsen']['Tuesday'][80][2]['31604-09']['all']['map'])

361

In [25]:
pickle.dump(all_equal_instances_objects, open("/content/drive/Shareddrives/Private Unlimited Drive #1/DDS/Analytics Project/Coding/Results/all_equal_instances_objects.p", "wb"))

## Calculate TSP Scenarios

In [20]:
all_equal_instances_objects = pickle.load(open("/content/drive/Shareddrives/Private Unlimited Drive #1/DDS/Analytics Project/Coding/Results/all_equal_instances_objects.p", "rb"))

In [21]:
all_complete_solver = pickle.load(open("/content/drive/Shareddrives/Private Unlimited Drive #1/DDS/Analytics Project/Coding/Results/all_complete_solver.p", "rb"))

In [26]:
# lists to store results

list_region = []
list_district = []
list_day = []
list_shipment_lvl = []
list_instance_number = []
list_volume_letter = []
list_volume_package = []
list_volume_others = []
list_volume_all = []
list_po_with_demand = []
list_po_all = []
list_instance_solver = []
list_skip_solver = []
list_skip_default = []
list_sim_instance_skipsolver = []
list_sim_instance_skipdefault = []
list_sim_skipdefault_skipsolver = []

In [31]:
%%time

## iterate through all regions ##
# for region in all_equal_instances_objects.keys():
for region in ['Warmsen']:
  data_region = all_equal_instances_objects[region]
  ## iterate through all days ##
  # for day in data_region.keys():
  for day in ['Monday', 'Friday']:
    data_day = data_region[day]
    ## iterate through all demand increase rates ##
    # for rate_increase in data_day.keys():
    for rate_increase in [5,10,50,80]:
      data_rate_increase = data_day[rate_increase]
      ## iterate through all instance number identifieres ##
      for instance in data_rate_increase.keys():
        data_instance = data_rate_increase[instance]
        ## iterate through all districts ##
        # for district in data_instance.keys():
        for district in ['31603-06', '31600-01']:
          data_district = data_instance[district]['all']
          start = datetime.datetime.now()
          
          ## solve TSP ##
          tsp = TSP(data_district['dm'])

          solver_info = {}
          solver_info['tour'], solver_info['cost'] = tsp.solve()
          solver_info['tour'] = map_actual_route(solver_info['tour'], data_district['map'])
          solver_info['tour'] = arrange_tour(solver_tour = solver_info['tour'],
                                                      start_end_points = data_district['start_end_points'])
          # print('instance solver ok')
          # data_district['solver'] = solver_info

          ## skip from complete default DP route ##
          skip_from_default = {}
          skip_from_default['route'], skip_from_default['cost'] = new_tour_sequence_fixed(complete_tour = all_default_tours[region][district]['default_route'],
                                                                                          dm_instance = data_district['dm'],
                                                                                          mapping_pp_needed = data_district['map'])
          # print('skip_from_default ok')
          ## skip from complete route solver ##
          skip_from_solver = {}
          skip_from_solver['route'], skip_from_solver['cost'] = new_tour_sequence_fixed(complete_tour = all_complete_solver[region][district]['tour'],
                                                                                          dm_instance = data_district['dm'],
                                                                                          mapping_pp_needed = data_district['map'])
          # print('skip_from_solver ok')
          ## similarity : instance solver vs skip from complete route solver ##
          num_changed_pred, sim_instance_skipsolver = find_similarity_route(solver_info['tour'], skip_from_solver['route'])
          # print('sim_instance_skipsolver ok')
          ## similarity : instance solver vs skip from complete default DP route ##
          num_changed_pred, sim_instance_skipdefault = find_similarity_route(solver_info['tour'], skip_from_default['route'])
          # print('sim_instance_skipdefault ok')
          ## similarity : skip from complete default DP route vs skip from complete route solver ##
          num_changed_pred, sim_skipdefault_skipsolver = find_similarity_route(skip_from_default['route'], skip_from_solver['route'])
          # print('sim_instance_skipsolver ok')

          ## store data ##
          list_region.append(region)
          list_district.append(district)
          list_day.append(day)
          list_shipment_lvl.append(rate_increase)
          list_instance_number.append(instance)
          list_volume_letter.append(data_district['volume']['letters'])
          list_volume_package.append(data_district['volume']['packages'])
          list_volume_others.append(data_district['volume']['others'])
          list_volume_all.append(data_district['volume']['all'])
          list_po_with_demand.append(len(data_district['dm'])-2)
          list_po_all.append(len(all_default_tours[region][district]['default_route'])-2)
          list_instance_solver.append(solver_info['cost'])
          list_skip_solver.append(skip_from_solver['cost'])
          list_skip_default.append(skip_from_default['cost'])
          list_sim_instance_skipsolver.append(round(sim_instance_skipsolver, 2))
          list_sim_instance_skipdefault.append(round(sim_instance_skipdefault, 2))
          list_sim_skipdefault_skipsolver.append(round(sim_instance_skipsolver, ))

          ## log ##
          print('generated :', region, district, day, ': demand increase rate', rate_increase, '% :', 'instance number', instance,
                ': time needed', datetime.datetime.now() - start)


generated : Warmsen 31603-06 Monday : demand increase rate 5 % : instance number 1 : time needed 0:00:00.188720
generated : Warmsen 31600-01 Monday : demand increase rate 5 % : instance number 1 : time needed 0:00:00.114396
generated : Warmsen 31603-06 Monday : demand increase rate 5 % : instance number 2 : time needed 0:00:00.080972
generated : Warmsen 31600-01 Monday : demand increase rate 5 % : instance number 2 : time needed 0:00:00.156783
generated : Warmsen 31603-06 Monday : demand increase rate 10 % : instance number 1 : time needed 0:00:00.218453
generated : Warmsen 31600-01 Monday : demand increase rate 10 % : instance number 1 : time needed 0:00:00.131458
generated : Warmsen 31603-06 Monday : demand increase rate 10 % : instance number 2 : time needed 0:00:00.241027
generated : Warmsen 31600-01 Monday : demand increase rate 10 % : instance number 2 : time needed 0:00:00.500112
generated : Warmsen 31603-06 Monday : demand increase rate 50 % : instance number 1 : time needed 0:

In [33]:
# generate results into dataframe

result_df = pd.DataFrame(
    {
        'Region' : list_region,
     'District' : list_district,
     'Day' : list_day,
     'Shipment Level in %' : list_shipment_lvl,
     'Instance Number' : list_instance_number,
     'Volume Letters' : list_volume_letter,
     'Volume Packages' : list_volume_package,
     'Volume Others' : list_volume_others,
     'Volume All' : list_volume_all,
     'Total number of post objects with shipments' : list_po_with_demand,
     'Total number of all post objects' :  list_po_all,
     'TSP sequence default' : list_skip_default,
     'TSP sequence solver' : list_skip_solver,
     'TSP recalc' : list_instance_solver,
     'Similarity%_recalc_seqsolver' : list_sim_instance_skipsolver,
     'Similarity%_recalc_seqdefault' : list_sim_instance_skipdefault,
     'Similarity%_seqdefault_seqsolver' : list_sim_skipdefault_skipsolver
    }
)

In [34]:
result_df.head()

,Region,District,Day,Shipment Level in %,Instance Number,Volume Letters,Volume Packages,Volume Others,Volume All,Total number of post objects with shipments,Total number of all post objects,TSP sequence default,TSP sequence solver,TSP recalc,Similarity%_recalc_seqsolver,Similarity%_recalc_seqdefault,Similarity%_seqdefault_seqsolver
0,Warmsen,31603-06,Monday,5,1,56,20,0,76,69,422,9421.00,9238.58,8467,35.714286,17.142857,35.714286
1,Warmsen,31600-01,Monday,5,1,53,26,0,79,68,426,8775.15,9374.79,8476,34.285714,24.285714,34.285714
2,Warmsen,31603-06,Monday,5,2,50,14,0,64,55,422,7909.93,7632.38,7055,33.928571,12.500000,33.928571
3,Warmsen,31600-01,Monday,5,2,63,20,0,83,68,426,7809.33,8817.80,7595,47.142857,24.285714,47.142857
4,Warmsen,31603-06,Monday,10,1,53,31,0,84,70,422,8952.58,8907.80,7991,28.169014,14.084507,28.169014


In [35]:
# export to csv

result_df.to_csv("/content/drive/Shareddrives/Private Unlimited Drive #1/DDS/Analytics Project/Coding/Results/preliminary_results.csv")